In [34]:
import cv2
import numpy as np
from ultralytics import YOLO
import json
from pathlib import Path
from collections import defaultdict
from sklearn.cluster import KMeans
import pandas as pd
import random

In [9]:
VIDEO_PATH = "game_videos/10-12-2025/10-12-2025_full_game.mp4"
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

In [10]:
SAMPLE_RATE = 15 # process every Nth frame
CONF_THRESHOLD = 0.5 # detection confidence threshold

In [11]:
model = YOLO("yolov8n.pt") # nano version

In [12]:
detections_by_frame = []

In [13]:
cap = cv2.VideoCapture(VIDEO_PATH)

In [14]:
fps = cap.get(cv2.CAP_PROP_FPS)

In [16]:
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [21]:
print(f"Video: {fps} fps, {total_frames} frames, {total_frames/fps/60:.1f} minutes")
print(f"Will process ~{total_frames//SAMPLE_RATE} frames")

Video: 60.0 fps, 165482 frames, 46.0 minutes
Will process ~11032 frames


In [22]:
frame_count = 0

In [23]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    if frame_count % SAMPLE_RATE == 0:
        timestamp = frame_count / fps
        
        # Run detection
        results = model(frame, classes=[0], conf=CONF_THRESHOLD, verbose=False)
        boxes = results[0].boxes
        
        frame_data = {
            'frame_number': frame_count,
            'timestamp': timestamp,
            'players': []
        }
        
        # Extract each detected player
        for box in boxes:
            xyxy = box.xyxy[0].cpu().numpy()  # bounding box coords
            conf = float(box.conf[0])
            
            x1, y1, x2, y2 = map(int, xyxy)
            
            # Crop player region for color analysis
            player_crop = frame[y1:y2, x1:x2]
            
            if player_crop.size == 0:  # skip empty crops
                continue
            
            # Convert to HSV for better color analysis
            hsv_crop = cv2.cvtColor(player_crop, cv2.COLOR_BGR2HSV)
            
            # Focus on upper body (jersey region, not shorts/shoes)
            height = y2 - y1
            upper_third = hsv_crop[:height//3, :]
            
            # Calculate mean HSV values
            mean_hue = float(np.mean(upper_third[:, :, 0]))
            mean_sat = float(np.mean(upper_third[:, :, 1]))
            mean_val = float(np.mean(upper_third[:, :, 2]))
            
            # Also get mean BGR for reference
            bgr_crop = player_crop[:height//3, :]
            mean_bgr = bgr_crop.mean(axis=(0, 1))
            
            player_data = {
                'bbox': [x1, y1, x2, y2],
                'confidence': conf,
                'center': [(x1 + x2) // 2, (y1 + y2) // 2],
                'color_hsv': [mean_hue, mean_sat, mean_val],
                'color_bgr': [float(mean_bgr[0]), float(mean_bgr[1]), float(mean_bgr[2])],
                'team': None  # will assign after clustering
            }
            
            frame_data['players'].append(player_data)
        
        detections_by_frame.append(frame_data)
        
        if frame_count % (SAMPLE_RATE * 30) == 0:  # progress every ~30 processed frames
            print(f"Processed {frame_count}/{total_frames} frames ({frame_count/total_frames*100:.1f}%)")
    
    frame_count += 1

cap.release()

print(f"\nDetection complete. Found detections in {len(detections_by_frame)} frames")

# Team classification using K-means clustering on HSV values
print("\nClassifying teams based on jersey colors...")

Processed 0/165482 frames (0.0%)
Processed 450/165482 frames (0.3%)
Processed 900/165482 frames (0.5%)
Processed 1350/165482 frames (0.8%)
Processed 1800/165482 frames (1.1%)
Processed 2250/165482 frames (1.4%)
Processed 2700/165482 frames (1.6%)
Processed 3150/165482 frames (1.9%)
Processed 3600/165482 frames (2.2%)
Processed 4050/165482 frames (2.4%)
Processed 4500/165482 frames (2.7%)
Processed 4950/165482 frames (3.0%)
Processed 5400/165482 frames (3.3%)
Processed 5850/165482 frames (3.5%)
Processed 6300/165482 frames (3.8%)
Processed 6750/165482 frames (4.1%)
Processed 7200/165482 frames (4.4%)
Processed 7650/165482 frames (4.6%)
Processed 8100/165482 frames (4.9%)
Processed 8550/165482 frames (5.2%)
Processed 9000/165482 frames (5.4%)
Processed 9450/165482 frames (5.7%)
Processed 9900/165482 frames (6.0%)
Processed 10350/165482 frames (6.3%)
Processed 10800/165482 frames (6.5%)
Processed 11250/165482 frames (6.8%)
Processed 11700/165482 frames (7.1%)
Processed 12150/165482 frames

In [ ]:
# Collect all player color data
all_colors = []
player_indices = []  # track which frame and player each color belongs to

for frame_idx, frame_data in enumerate(detections_by_frame):
    for player_idx, player in enumerate(frame_data['players']):
        # Use HSV for clustering (more robust to lighting)
        all_colors.append(player['color_hsv'])
        player_indices.append((frame_idx, player_idx))

if len(all_colors) > 0:
    all_colors = np.array(all_colors)
    
    # Simple K-means with K=2 (two teams)
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    team_labels = kmeans.fit_predict(all_colors)
    
    # Assign team labels back to detections
    for idx, (frame_idx, player_idx) in enumerate(player_indices):
        detections_by_frame[frame_idx]['players'][player_idx]['team'] = int(team_labels[idx])
    
    # Print cluster centers for inspection
    print("\nTeam color profiles (HSV):")
    for team_id, center in enumerate(kmeans.cluster_centers_):
        count = np.sum(team_labels == team_id)
        print(f"Team {team_id}: H={center[0]:.1f}, S={center[1]:.1f}, V={center[2]:.1f} ({count} detections)")


Team color profiles (HSV):
Team 0: H=75.7, S=99.5, V=199.6 (40771 detections)
Team 1: H=101.6, S=60.7, V=224.2 (25678 detections)


In [25]:
# Save detections to JSON
output_path = OUTPUT_DIR / 'detections.json'
with open(output_path, 'w') as f:
    json.dump(detections_by_frame, f, indent=2)

print(f"\nSaved detections to {output_path}")


Saved detections to output/detections.json


In [27]:
rows = []

In [28]:
rows = []
for frame_data in detections_by_frame:
    for player in frame_data['players']:
        rows.append({
            'timestamp': frame_data['timestamp'],
            'frame': frame_data['frame_number'],
            'x': player['center'][0],
            'y': player['center'][1],
            'team': player['team'],
            'confidence': player['confidence'],
            'hue': player['color_hsv'][0],
            'saturation': player['color_hsv'][1],
            'value': player['color_hsv'][2]
        })

df = pd.DataFrame(rows)
csv_path = OUTPUT_DIR / 'detections_summary.csv'
df.to_csv(csv_path, index=False)

In [29]:
print(f"Saved summary CSV to {csv_path}")
print(f"\nTotal player detections: {len(df)}")
print(f"Team distribution:\n{df['team'].value_counts()}")

Saved summary CSV to output/detections_summary.csv

Total player detections: 66449
Team distribution:
team
0    40771
1    25678
Name: count, dtype: int64


In [30]:
df

,timestamp,frame,x,y,team,confidence,hue,saturation,value
0,0.0,0,2229,825,0,0.859100,48.846491,96.315840,206.280551
1,0.0,0,627,606,0,0.847719,58.906070,106.238066,201.568928
2,0.0,0,1482,243,0,0.834371,71.380000,79.789412,190.711647
3,0.0,0,494,75,1,0.631141,107.996599,89.767687,228.914626
4,0.0,0,2240,487,1,0.506192,100.590110,88.556410,225.989744
...,...,...,...,...,...,...,...,...,...
66444,2758.0,165480,460,740,0,0.797097,59.096232,107.420078,216.204391
66445,2758.0,165480,2115,244,0,0.700316,68.769213,75.630606,184.938472
66446,2758.0,165480,2378,160,0,0.685568,106.009841,81.462857,189.639683
66447,2758.0,165480,1904,151,0,0.675929,104.996482,103.580109,162.573713


In [33]:
detections_by_frame[1000]

{'frame_number': 15000,
 'timestamp': 250.0,
 'players': [{'bbox': [444, 180, 491, 318],
   'confidence': 0.7190278172492981,
   'center': [467, 249],
   'color_hsv': [107.56336725254394, 96.23959296947271, 222.3149861239593],
   'color_bgr': [156.73172987974098, 144.35060129509714, 222.25346901017576],
   'team': 1},
  {'bbox': [1645, 114, 1739, 360],
   'confidence': 0.6634374260902405,
   'center': [1692, 237],
   'color_hsv': [69.66645044110015, 68.63609237156201, 197.3663725998962],
   'color_bgr': [148.93824597820446, 172.1104047742605, 180.61468604047744],
   'team': 0},
  {'bbox': [2362, 253, 2427, 374],
   'confidence': 0.5651023983955383,
   'center': [2394, 313],
   'color_hsv': [60.495384615384616, 6.228076923076923, 246.16884615384615],
   'color_bgr': [244.90769230769232, 241.08153846153846, 245.81846153846155],
   'team': 1}]}

In [42]:
# Pick a random frame to visualize
frame_data = random.choice(detections_by_frame)
frame_num = frame_data['frame_number']

# Read that frame from video
cap = cv2.VideoCapture(VIDEO_PATH)
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
ret, frame = cap.read()
cap.release()

In [43]:
# Draw bounding boxes with team colors
for player in frame_data['players']:
    x1, y1, x2, y2 = player['bbox']
    team = player['team']
    
    # Team 0 = red, Team 1 = blue
    color = (0, 0, 255) if team == 0 else (255, 0, 0)
    
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
    cv2.putText(frame, f"T{team}", (x1, y1-10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)



In [46]:
cv2.imwrite('output/visualized_frame.jpg', frame)
print(f"Saved visualization to output/visualized_frame.jpg")

Saved visualization to output/visualized_frame.jpg
